In [13]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
import random
from copy import deepcopy
random.seed(123)

In [14]:
#DATA EXTRACTION
data=pd.read_csv('/Users/michaelfilletti/Desktop/Uni/AI/Machine Learning/Assignment/Datasets/Absenteeism_at_work_AAA/Absenteeism_at_work.csv',sep=';')
# Get one hot encoding of columns B
a=['Reason for absence', 'Month of absence', 'Day of the week', 'Seasons', 'Education', 'Disciplinary failure', 'Social drinker', 'Social smoker']

#ONE HOT ENCODING
for i in range(0,len(a)):
    one_hot = pd.get_dummies(data[a[i]])
    data = data.drop(a[i],axis = 1)
    data = data.join(one_hot)
    s=a[i]
    for j in range(0,np.shape(one_hot)[1]+1):
        data.rename(columns={j:a[i]+str(j)}, inplace=True)

#FEATURE SELECTION - PCC
data2=data.drop(['ID', 'Absenteeism time in hours'], axis=1)
y = data.iloc[:,12].values
N=np.shape(data)[0]
sY=np.sum(data['Absenteeism time in hours'].values)
sY2=np.sum(np.square(data['Absenteeism time in hours'].values))
PCC=np.zeros(np.shape(data2)[1])
for i in range(0,np.shape(data2)[1]):
    var=data2.iloc[:,i].values
    sXY=np.sum(np.multiply(var, y))
    sX=np.sum(var)
    sX2=np.sum(np.square(var))
    data.iloc[:,i]
    PCC[i]=abs(N*sXY-(sX*sY))/np.sqrt((N*sX2-sX**2)*(N*sY2-sY**2))
ind=np.argsort(-PCC)


selected_features=15
index=ind[0:selected_features]
X=data2.iloc[:,index]

normalized_X=(X-X.mean())/X.std() #normalized X
ydf=pd.DataFrame(y,columns=['y'])
dataset=pd.concat([normalized_X, ydf], axis=1)

In [15]:
#SMO STRUCTURE
#Declaration of variables and various kernel options (lin or rbf)
class SMOStructure:
    # init the structure with parameters
    def __init__(self, X, y, C, Cst, toler, kernel, gamma, epsilon, r, d):
        self.X = np.mat(X) #independent data inputted by user
        self.y = np.mat(y).T #dependent data inputted by user
        self.y1=0 #y value of corresponding index
        self.chi = 0 #chi value
        self.C = C #inputted by user
        self.Cst = Cst #inputted by user
        self.epsilon = epsilon #inputted by user
        self.tol = toler #inputted by user
        self.gamma=gamma #inputted by user (used for rbf, poly and sigmoid)
        if gamma<=0:
            pass
        self.r = r #inputted by user (used for poly and sigmoid)
        self.d = d #inputted by user (used for poly)
        self.test=0 #dummy variable used to debug
        self.kernel = kernel #inputted by user
        self.m = np.shape(X)[0] #number of rows of training set
        self.n = np.shape(X)[1] #number of cols of training set
        self.alphas = np.mat(np.zeros((self.m, 1))) #Initial value of alpha paras
        self.alphast = np.mat(np.zeros((self.m, 1))) #Initial value of alpha paras
        self.beta = 0 #Initial beta value
        self.r1=0 #Initial KKT condn value
        self.b = 0 #Initial value of bias para
        self.e_cache = np.mat(np.zeros(self.m)).T #Error cache: holds memory of error
        self.K = np.mat(np.zeros((self.m, self.m)))
        self.gamma1=0
        # init kernel cache matrix: lin or rbf
        if kernel == 'lin':
            self.K = self.X * self.X.T
        elif kernel == 'rbf':
            self.K = np.mat(np.zeros((self.m, self.m)))
            for i in range(self.m):
                for j in range(self.m):
                    self.K[i, j] = (self.X[i] - self.X[j]) * (self.X[i] - self.X[j]).T
                    self.K[i, j] = np.exp(self.K[i, j] * (-1 * self.gamma))
        elif kernel == 'poly':
            self.K = (self.gamma * self.X * self.X.T + self.r)**self.d
        elif kernel == 'sigmoid':
            self.K = np.tanh(self.gamma * self.X * self.X.T + self.r)
        else:
            pass

In [16]:
#function takestep
#This is the algorithm for REGRESSION
def take_step(i1, i2, smo):
    
    #Declaring variables
    alpha1 = smo.alphas[i1]
    alphast1 = smo.alphast[i1]
    y1 = smo.y[i1]
    total=0
    for j in range(0,len(smo.X)):
        total += (smo.alphas[i1] - smo.alphast[i1]) * smo.K[j,i1]
    E1 = total + smo.b - y1 #f(x)-y
    
    #Declaring other variables 
    alpha2 = smo.alphas[i2]
    alphast2 = smo.alphast[i2]
    y2 = smo.y[i2]
    E2 = smo.e_cache[i2]
    smo.gamma1=(alpha1-alphast1)+(alpha2-alphast2)
    
    #Defining L,H,l,h
    L = max(smo.gamma1-smo.C, -smo.Cst)
    H = min(smo.gamma1+smo.Cst, smo.C)
    if L == H:
        return 0
    l=min(smo.gamma1,0)
    h=max(smo.gamma1,0)
    
    #Chi (kernel sum and subtraction)
    smo.chi = smo.K[i1, i1] + smo.K[i2, i2] - 2*smo.K[i1, i2]
    #Updating a2
    if smo.chi > 0:
        #a2 = alpha2 + y2*(E1-E2)/chi
        beta0 = (alpha1-alphast1) + (E1-E2)/smo.chi
        betap = beta0 - (2*smo.epsilon/smo.chi)
        betan = beta0 + (2*smo.epsilon/smo.chi)
        
        beta=max(min(beta0,h),l) #Let beta be the point that lies within the interval (i.e. beta0, h or l)
        if beta == h: #If beta is h, this means beta0 it is not in I0, so we move to betap
            beta = max(min(betap,H),h) #Again, we find the point within the interval
        elif beta == l: #If beta is l, this means it is not in I0
            beta = max(min(betan,l),L) #We find the point within the interval
    elif (E1-E2) < 0:
        beta=h
        if (E1-E2)+2*smo.epsilon < 0:
            beta=H
    else:
        beta=l
        if (E1-E2)-2*smo.epsilon > 0:
            beta=L
    
    #Converging condition
    #if abs(beta-(alpha1-alphast1)) < smo.epsilon*(smo.epsilon+abs(beta)+alpha1+alphast1):
    #    return 0
    
    #Updating alpha
    alpha1 = max(beta,0)
    alphast1 = max(-beta,0)
    alpha2 = max(0,smo.gamma1-beta)
    alphast2=max(0,beta-smo.gamma1)
    
    #Updating b
    if alpha1 > 0 and alpha1 < smo.C:
        bnew =  y1 - (smo.K[i1, i1] * (smo.alphas[i1] - smo.alphast[i1])) - smo.epsilon
    elif alphast1 > 0 and alphast1 < smo.Cst:
        bnew = y1 - (smo.K[i1, i1] * (smo.alphas[i1] - smo.alphast[i1])) + smo.epsilon
    else:
        b1 = y1 - (smo.K[i1, i1] * (smo.alphas[i1] - smo.alphast[i1])) - smo.epsilon
        b2 = y1 - (smo.K[i1, i1] * (smo.alphas[i1] - smo.alphast[i1])) + smo.epsilon
        bnew = (b1 + b2) / 2
    
    #Declaring the new b, a1 and a2
    smo.b = bnew
    smo.alphas[i1] = alpha1
    smo.alphast[i1] = alphast1
    smo.alphas[i2] = alpha2
    smo.alphast[i2] = alphast2
    
    
    #Updating cache (y_1-f(x_1), y_2-f(x_2),...)
    total=0
    for i in range(smo.m):
        if (smo.alphas[i] > 0) and (smo.alphas[i] < smo.C):
            for j in range(0,len(smo.X)):
                total += (smo.alphas[i] - smo.alphast[i])*smo.K[j,i]
            smo.e_cache[i] = total + smo.b - smo.y[i] #Error
    return 1

In [17]:
#Testing Heuristics
def examine_example(i1, smo):
    #Declare variables based on i1
    smo.y1 = smo.y[i1]
    alpha1 = smo.alphas[i1]
    alphast1 = smo.alphast[i1]
    H=min(smo.gamma1+smo.Cst,smo.C) #initial value for H
    total=0
    
    for j in range(0,len(smo.X)):
        total += (smo.alphas[i1] - smo.alphast[i1])*smo.K[j,i1]
    E1 = total + smo.b - smo.y1
    #Set elements of e_cache
    smo.e_cache[i1] = E1
    #r1 = alpha1*max(0,E1+smo.epsilon)+alphast1*max(0,-E1+smo.epsilon)+(smo.C-alpha1)*max(0,E1+smo.epsilon)+(smo.Cst-alphast1)*max(0,E1-smo.epsilon) 
    #r1 represents KKT
    smo.r1=H*alpha1*max(0,E1+smo.epsilon)+H*alphast1*max(0,smo.epsilon-E1)+H*(smo.C-alpha1)*max(0,smo.epsilon-E1)+H*(smo.Cst-alphast1)*max(0,E1-smo.epsilon)
    
    #---Identifying if KKT conditions are satisfied
    #if r1 lies outside tolerance region and 
    if(smo.r1>smo.tol):
        #Only if there is a nonbound and nonzero alpha, we find j
        if (smo.alphas[i1] < smo.C) or (smo.alphas[i1] > 0):
            
            # heuristic 1: find the max deltaE to know which j to select
            max_delta_E = 0
            i2 = -1
            #Cover all training set
            for i in range(smo.m):
                #If it lies within the bound
                if smo.alphas[i] > 0 and smo.alphas[i] < smo.C:
                    E2 = smo.e_cache[i]
                    delta_E = abs(E1 - E2)
                    if delta_E > max_delta_E:
                        max_delta_E = delta_E #update max_delta_E
                        i2 = i #Let i2 be the i with the greatest max_delta_E
            #if i1 >= 0:
            if take_step(i1, i2, smo)==1:
                return 1
        
        # heuristic 2: find the suitable i2 on border at random
        #---Loop over all non zero and non C alpha
        random_index = np.random.permutation(smo.m) #Randomly index training set
        for i in random_index:
            if smo.alphas[i] > 0 and smo.alphas[i] < smo.C:
                if take_step(i1, i, smo)==1:
                    return 1
            
        
        # heuristic 3: find the suitable i1 at random on all alphas
        #---Loop over all possible i
        #random_index = set(np.random.permutation(smo.m)) - set(indices)
        for i in random_index:
            if take_step(i1, i, smo)==1:
                return 1
    return 0

In [18]:
#Main SMO Process
#Set number of iterations here, and sets how the program works
def SMORegression(smo, max_iter=20):
    #Set variables
    num_changed = 0
    examine_all = 1
    passes = 0
    #Sets number of iterations to loop
    #Bounds number of passes the program can make
    while(passes <= max_iter):
        num_changed = 0 #Reset num_changed to 0
        
        if (examine_all == 1): #If examine_all is set to 1
            for i1 in range(smo.m): #Loop i1 over whole range
                num_changed += examine_example(i1, smo) #Run examine_example
        else:
            for i1 in range(smo.m): #If examine_all is not set to 1
                if (smo.alphas[i1] > 0) and (smo.alphas[i1] < smo.C): #Only carry out for those not on bound
                    num_changed += examine_example(i1, smo) #Run examine_example
        
        if (num_changed == 0):
            passes += 1 #Increment passes by 1 if num_changed is 0
        
        if (examine_all == 1):
            examine_all = 0
        elif (num_changed == 0):
            examine_all = 1

In [19]:
#---------K-FOLD CROSS-VALIDATION---------
#Split a dataset into k folds
#the original sample is randomly partitioned into k equal sized subsamples. 
#Of the k subsamples, a single subsample is retained as the validation data 
#for testing the model, and the remaining k − 1 subsamples are used as training data. 
#The cross-validation process is then repeated k times (the folds),
#with each of the k subsamples used exactly once as the validation data.
def cross_validation_split(data, n_folds):
    dataset_split = list() #Create list containing dataset split
    dataset_copy = list(data) #Convert data to list
    fold_size = int(len(data) / n_folds) #Fold size = total length of data / no of folds
    for i in range(n_folds):
        fold = list()
        while len(fold) < fold_size: #Adding data into each fold
            index = random.randrange(0,len(dataset_copy)) #generate random number to input into fold
            fold.append(dataset_copy.pop(index)) #input corresponding data point into fold
        dataset_split.append(fold) #append to split data
    return dataset_split

# Evaluate an algorithm using a cross validation split
def evaluate_algorithm(dataset, n_folds, C, Cst, toler, kernel, gamma, epsilon, r, d):
    #folds are the subsamples used to train and validate model
    folds = cross_validation_split(dataset, n_folds) #call previous function
    scores = list() #list of scores
    i=0
    #for each subsample
    for i in range(0,n_folds):
        #create a copy of the data
        test_set=folds[i]
        train_set = deepcopy(folds) #Training set
        #remove the given subsample
        train_set.pop(i) #Remove one of the folds
        train_set = sum(train_set, []) #Make two dimensional
        #init a test set
        #test_set = list() #empty list
        
        #add each row in a given subsample to the test set
        #for row in fold:
        #    row_copy = list(row)
        #    test_set.append(row_copy) #appending each row to the test set
        #    row_copy[-1] = None
        
        #get predicted labels
        ytrain=[item[-1] for item in train_set]
        xtrain=[item[:-2] for item in train_set]
        ytest=[item[-1] for item in test_set]
        xtest=[item[:-2] for item in test_set]
        smo = SMOStructure(xtrain, ytrain, C, Cst, toler, kernel, gamma, epsilon, r, d) #applying the algorithm
        SMORegression(smo)
        K=np.mat(np.zeros((len(train_set), len(test_set))))
        ypred=np.zeros(len(test_set))
        
        xtrainm=np.concatenate(xtrain).reshape(len(train_set),np.shape(dataset)[1]-2)
        xtestm=np.concatenate(xtest).reshape(np.shape(dataset)[1]-2,len(test_set))
        K = (xtrainm).dot(xtestm)
        for i in range(0,len(test_set)):
            total=0
            for j in range(0,len(train_set)):
                total += (smo.alphas[j] - smo.alphast[j])*K[j,i]
            ypred[i] = total + smo.b
            #get actual labels
        
        #actual = ytest #get y values
        #compare accuracy
        accuracy = mean_squared_error(ytest, ypred) #compare y values to actual values (MSE)
        #add it to scores list, for each fold
        scores.append(accuracy)
        #print(scores)
    #return all accuracy scores
    return (scores, np.mean(scores))

In [20]:
#RUNNING THE ALGORITHM (using k-fold cross-validation)
cross_validation_split(np.array(dataset), 3)
evaluate_algorithm(np.array(dataset), n_folds=3, C=1000, Cst=1000, toler=0.001, kernel='rbf', gamma=1, epsilon=10, r=10, d=1)

([113440727.86272976, 255.20731707317069, 323230650.6073509],
 145557211.22579923)

In [43]:
#PACKAGE PERFORMANCE
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)
clf = SVR(C=0.01, epsilon=0.01,kernel='rbf',gamma=5)
clf.fit(X_train, y_train) 
ypredskl=clf.predict(X_test)
msesk=mean_squared_error(y_test, ypredskl)
print('MSE of sklearn Model:',msesk)

#Using K-fold Cross Validation
scores = cross_validate(clf, X, y, cv=3,scoring='neg_mean_squared_error',return_train_score=True)
print('Average KCV score',-np.mean(pd.DataFrame(scores).iloc[:,2]))

MSE of sklearn Model: 213.97230559991613
Average KCV score 190.5349938966664
